# 03: Perform Data Cleaning
This notebook drops redundant features, strips whitespace, handles missing values, removes duplicates and impossible records.

## 3.1: Import Library

In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", None)

## 3.2: Load CSV File

In [2]:
df = pd.read_csv('../data/processed/UNSW-NB15.csv', na_values=['-'], low_memory=False)

## 3.3: Remove Redundant Features

In [3]:
features_to_drop = [
    'srcip',  # Makes model learn environment not behaviour
    'dstip',  # Makes model learn environment not behaviour
    'sport',  # Randomly assigned by OS
    'dsport', # 'service' already captures protocol-level destination port meaning in a cleaner categorical form
    'Stime',  # Makes model learn environment not behaviour
    'Ltime',  # Makes model learn environment not behaviour
    'tcprtt', # Sum of 'synack' + 'ackdat' (does not provide information)
    'stcpb',  # TCP sequence numbers are randomly initialised
    'dtcpb'   # TCP sequence numbers are randomly initialised
]
df.drop(columns=features_to_drop, inplace=True)

## 3.4: Strip Whitespace

In [4]:
# Strip whitespace in column headers
df.columns = df.columns.str.strip()
# Strip whitespace in string features
columns = df.select_dtypes(include=['object', 'string']).columns
df[columns] = df[columns].apply(lambda x: x.str.strip())

In [5]:
# Check if whitespace in 'attack_cat' column has been removed
df["attack_cat"].value_counts()

attack_cat
Generic           215481
Exploits           44525
Fuzzers            24246
DoS                16353
Reconnaissance     13987
Analysis            2677
Backdoor            1795
Shellcode           1511
Backdoors            534
Worms                174
Name: count, dtype: int64

## 3.5: Handle Missing Values

In [6]:
# Check which features contain missing values
missing_values = df.isna().sum()
missing_values[missing_values > 0]

service             1246397
ct_flw_http_mthd    1348145
is_ftp_login        1429879
attack_cat          2218764
dtype: int64

In [7]:
df[df["is_ftp_login"].isnull() & (df["service"] != 'ftp')]

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
1087203,tcp,FIN,0.121478,258,172,252,254,0,0,NaN,...,,1,1,1,1,1,1,1,NaN,0
1087204,tcp,FIN,0.649902,734,42014,62,252,2,17,NaN,...,,43,6,1,1,1,1,2,NaN,0
1087205,tcp,FIN,1.623129,364,13186,62,252,1,6,NaN,...,,7,6,2,2,1,1,3,NaN,0
1087207,tcp,FIN,0.449454,534,268,254,252,2,1,NaN,...,,43,39,2,2,2,1,40,NaN,0
1087208,tcp,FIN,0.380537,534,268,254,252,2,1,NaN,...,,43,39,2,2,2,1,40,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540039,tcp,CON,0.564998,14106,772406,31,29,2,262,http,...,,1,1,4,2,2,2,2,NaN,0
2540041,tcp,CON,0.564998,14106,772406,31,29,2,262,NaN,...,,2,1,4,2,2,2,2,NaN,0
2540042,tcp,FIN,0.087306,320,1828,31,29,1,2,ftp-data,...,,1,2,3,3,1,1,3,NaN,0
2540045,tcp,CON,2.200934,3498,166054,31,29,2,57,http,...,,1,1,2,4,2,2,2,NaN,0


In [8]:
df['service'].value_counts()

service
dns         781668
http        206273
ftp-data    125783
smtp         81645
ftp          49090
ssh          47160
pop3          1533
dhcp           172
ssl            142
snmp           113
radius          40
irc             31
Name: count, dtype: int64